# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [ ]:
#!pip -q install ./python

The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [13]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [14]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

print(f'brain_name is {brain_name}')

brain_name is TennisBrain


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print('The state for the 2nd agent looks like:', states[1])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [8]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

KeyboardInterrupt: 

When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [7]:
from maddpg import MADDPGAgent
from collections import deque
from workspaceUtils import keep_awake
import torch

In [8]:
agent = MADDPGAgent(state_size=state_size, action_size=action_size, seed=0)

In [9]:
def train(n_episodes=10000, max_t=1000):
    """
    INPUT
    - n_episodes (int): max number of training episodes
    - max_t (int): max time steps for each training episode
    """
    scores = [] # list of average score of all agents for each training episode 
    scores_deque = deque(maxlen=100) # last 100 averaged score for all agents
    scores_mean = [] # list of average score for all agents for 100 consecutive episodes
    
    for i in keep_awake(range(1, n_episodes+1)):
        # reset env
        env_info = env.reset(train_mode=True)[brain_name]
        
        # to store the score for each agent for this episode, ini to 0
        agents_score = np.zeros(num_agents)
        
        # ini state
        states = env_info.vector_observations
        
        # reset the noise process
        agent.reset()
        
        for t in range(max_t):
            # agents take actions according to DDPG policy
            actions = agent.act(states)
            
            # Upon receiving actions, env transits to the next state and provides rewards
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            # agent learn from experience tuples
            agent.step(states, actions, rewards, next_states, dones)
            
            # update states
            states = next_states
            
            # accumulative score for each agent
            agents_score += rewards
            
            # check if any of the agent reach the end of episode
            if np.any(dones):
                break
                
        # take the max scores of all agent for this episode
        score_for_this_episode = np.max(agents_score)
        
        scores_deque.append(score_for_this_episode)
        
        final_score = np.mean(scores_deque)
        
        scores.append(score_for_this_episode)
        
        if i % 100 == 0:
            print('\rEpisode {:}\t Average Score {:.2f}'.format(i, final_score, end=''))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor1.pth')
            torch.save(agent.critic_local.state_dict(),'checkpoint_critic1.pth')
        if i >= 100:
            scores_mean.append(final_score)
        if final_score >= 0.5:
            print(f'\nEnvironment solved in {i} episodes. \tAverage Score {final_score}')
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor1.pth')
            torch.save(agent.critic_local.state_dict(),'checkpoint_critic1.pth')
            break
    return scores, scores_mean, agent

In [10]:
scores, scores_mean, agent = train()

Episode 100	 Average Score 0.01
Episode 200	 Average Score 0.01
Episode 300	 Average Score 0.02
Episode 400	 Average Score 0.03
Episode 500	 Average Score 0.01
Episode 600	 Average Score 0.01
Episode 700	 Average Score 0.00
Episode 800	 Average Score 0.00
Episode 900	 Average Score 0.00
Episode 1000	 Average Score 0.00
Episode 1100	 Average Score 0.00
Episode 1200	 Average Score 0.01
Episode 1300	 Average Score 0.02
Episode 1400	 Average Score 0.03
Episode 1500	 Average Score 0.01
Episode 1600	 Average Score 0.03
Episode 1700	 Average Score 0.23
Episode 1800	 Average Score 0.29
Episode 2000	 Average Score 0.31
Episode 2100	 Average Score 0.22
Episode 2200	 Average Score 0.09
Episode 2300	 Average Score 0.13
Episode 2400	 Average Score 0.22
Episode 2500	 Average Score 0.28
Episode 2600	 Average Score 0.25
Episode 2700	 Average Score 0.06
Episode 2800	 Average Score 0.36
Episode 2900	 Average Score 0.26
Episode 3000	 Average Score 0.17
Episode 3100	 Average Score 0.13
Episode 3200	 Avera

NameError: name 'score_mean' is not defined

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(dscores)+1), dscores)
plt.title('MADDPG agent')
plt.ylabel('Score')
plt.xlabel('Episode')
plt.show()

# Test the trained agents

NOTE: the following codes are saved to the file `test.py`.

In [ ]:
!pip -q install ./python

#load packages
from unityagents import UnityEnvironment
from maddpg import MADDPGAgent
import numpy as np
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

# instantiate env
env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name]

# instantiate agent
agent = MADDPGAgent(state_size=env_info.vector_observations.shape[1], action_size= brain.vector_action_space_size, seed=1)

# Load trained model weights
agent.actor_local.load_state_dict(torch.load('checkpoint_actor1.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic1.pth'))

num_agents = len(env_info.agents)
env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
# initialize the score (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()